In [1]:
import pandas as pd
import numpy as np

In [2]:
#  данные 
xls = pd.ExcelFile('объект.xlsx')
object_ = pd.read_excel(xls, 'объекты')
time = pd.read_excel(xls, 'время')

In [3]:
object_ = object_.dropna() # удаляем nan

In [4]:
# Имена сотрудников – индекс
object_ = object_.set_index('сотрудник')
time = time.set_index('сотрудник')

In [5]:
object_

,2022-09-12 00:00:00,2022-09-13 00:00:00,2022-09-14 00:00:00,2022-09-15 00:00:00
сотрудник,,,,
Иванов,2569 км,2569 км,ст.Новая,ст.Новая
Петров,2569 км,2569 км,ст.Новая,ст.Новая
Кузнецов,2569 км,ст. Первая,ст. Первая,ст.Первая


In [6]:
time

,2022-09-12 00:00:00,2022-09-13 00:00:00,2022-09-14 00:00:00,2022-09-15 00:00:00
сотрудник,,,,
Иванов,8,8,8,8
Петров,12,8,12,8
Кузнецов,12,8,12,8


In [7]:
# Преобразуем таблицы 
object_ = object_.stack().rename_axis(('Имя','Дата')).reset_index(name='Объект') 
time = time.stack().rename_axis(('Имя','Дата')).reset_index(name='Час') 

In [8]:
# Объединяем 
join = object_.merge(time, how = 'inner', left_on=['Имя', 'Дата'], right_on=['Имя', 'Дата'])
join = join.groupby(['Дата', 'Объект'])['Час'].agg(['sum'])
join

sum
Дата       Объект         
2022-09-12 2569 км      32
2022-09-13 2569 км      16
           ст. Первая    8
2022-09-14 ст. Первая   12
           ст.Новая     20
2022-09-15 ст.Новая     16
           ст.Первая     8

In [9]:
# Сводная таблица 
table = pd.pivot_table(join, values='sum', index=['Объект'],
                    columns=['Дата'])
table

Дата,2022-09-12,2022-09-13,2022-09-14,2022-09-15
Объект,,,,
2569 км,32.0,16.0,NaN,NaN
ст. Первая,NaN,8.0,12.0,NaN
ст.Новая,NaN,NaN,20.0,16.0
ст.Первая,NaN,NaN,NaN,8.0
